Ensure you have 'embeddings.pkl' generated

In [1]:
import pandas as pd
import pickle
import re
import string
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/shruthihariharan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
emb = pickle.load(open('embeddings.pkl', 'rb'))##can replace with embeddings from fine-tuned model
sentences, embeddings = emb['sentences'], emb['embeddings']

In [8]:
df = pd.read_excel('full-unlabelled.xlsx')
df = df.dropna()

In [ ]:
def preprocess(op):
    txt = op.split("text\':")
    if len(txt)>1:
      txt = txt[1]
    else:
      return ""
    txt = re.sub('\\n',' ',txt)#txt.split('\\n')
    #txt = " ".join(txt)    
    txt = "".join(c for c in txt if c not in string.punctuation)
    words = (txt.split())
    words = [w for w in words if w.lower() not in stop_words and len(w) > 3][:512]
    final_txt = " ".join(words)    
    return final_txt

In [ ]:
df['opinion_text'] = df['opinions'].apply(lambda x: preprocess(x))
df = df.dropna()

In [ ]:
pip install faiss-cpu

Building Index

In [11]:
import faiss                   # make faiss available
index = faiss.IndexFlatL2(embeddings.shape[1])   # build the index
print(index.is_trained)
index.add(embeddings)                  # add vectors to the index
print(index.ntotal)

True
144112


In [9]:
# 4225064 and 6050740
df[df['id'] == 6050740]

,id,main_id,citation1,opinions
128884,6050740,6050740,48 Cal. 4th 564,"[{'text': 'Opinion\nCHIN, J.\nProposition 115 ..."


In [6]:
embeddings[98438].shape

(384,)

Searching similar documents within the data: Insert the row of any case from the full_unlabelled.xlsx file

In [12]:
k = 4                          # we want to see 4 nearest neighbors
# D, I = index.search(embeddings[:5], k) # sanity check
# print(I)
# print(D)
D, I = index.search(embeddings[128884].reshape(1,-1), k)     # actual search
print(I)                   # neighbors of the 5 first queries
print(I)       

[[128884  43084 122988  88530]]
[[128884  43084 122988  88530]]


In [15]:
sentences[88530]

'[{\'text\': "CARTER, J.\\nThis is a proceeding in mandamus in which petitioner seeks to compel respondent, Secretary of State, to take the various steps required of him by law to prepare for submission to the electorate at the next appropriate election a measure to repeal article XXV of the Constitution which was added at the general election in November of 1948. (Article XXV provides for pensions for the needy aged and blind, and as will hereafter more fully appear, is commonly referred to as Proposition 4, that being its position on the election ballot in 1948.) Compliance has been had with all the steps necessary for the repeal measure concerning petitions for an initiative constitutional amendment. Petitioner is the official proponent of the repealing measure. Heretofore, the present opponents of the repeal measure and the former proponents of article XXV commenced a proceeding in the Superior Court of Sacramento County to obtain a writ of mandate ordering respondent here to refra

In [12]:
for i in I[1]:
    print(sentences[i])
    print('\n\n')

[{'text': "Opinion\nGAUT, J.\n1. Introduction\nPlaza Freeway Limited Partnership (plaintiff) and First Mountain Bank (defendant) are successors in interest to the original landlord and tenant of commercial real property under a 25-year lease agreement. The parties were unable to determine the termination date of the ground lease. Nevertheless, at the time of plaintiff’s purchase of the property, defendant signed and delivered an estoppel certificate which provided a lease termination date of October 31, 1998. The trial court found the date on the estoppel certificate incorrect, and instead decided that the termination date was June 30, 1999. The court concluded that, based on the June 1999 termination date and the terms of the original lease agreement, defendant’s exercise of its option to renew on January 26, 1998, was timely. Thus, the court found defendant in lawful possession of the property and not guilty of an unlawful detainer.\nOn appeal, plaintiff claims the doctrine of “estop